# Feature Transforms

> This module defines a comprehensive feature transformation framework for data preprocessing in Snowflake, providing configurable and composable data transformations with built-in validation. It implements various transform types (window operations, null filling, date calculations, moving and cumulative aggregations) all following a consistent Transform protocol, and supports data quality checks through a ValidationMixin that enforces constraints on null ratios, cardinality, and data types.

In [1]:
#| default_exp transforms

In [2]:
#| export
from __future__ import annotations
from typing import Union, List, Callable, Optional, Protocol, Dict
from dataclasses import dataclass
from fastcore.basics import listify
import snowflake.snowpark.functions as F
from snowflake.snowpark import DataFrame, Window
from pydantic import BaseModel, Field, field_validator
import pandas as pd


# Import our new modules
from snowflake_feature_store.exceptions import ValidationError
from snowflake_feature_store.logging import logger
from snowflake_feature_store.config import BaseModel

In [3]:
#| export
class TransformConfig(BaseModel):
    """Configuration for data transformations"""
    name: str
    description: Optional[str] = None
    validate_output: bool = Field(True, description="Whether to validate transform output")
    
    # Validation settings
    null_threshold: float = Field(0.1, ge=0, le=1, description="Maximum allowed null ratio")
    cardinality_threshold: Optional[int] = Field(None, description="Maximum distinct values")
    expected_types: Optional[List[str]] = Field(
        None, 
        description="List of acceptable types (e.g., ['DECIMAL', 'DOUBLE', 'NUMBER'] for numeric)"
    )

    @field_validator('expected_types')
    @classmethod
    def validate_types(cls, v):
        if v is not None:
            valid_types = {'DECIMAL', 'DOUBLE', 'NUMBER', 'INT', 'LONG', 'STRING', 'BOOLEAN', 'DATE', 'TIMESTAMP'}
            for t in v:
                if t.upper() not in valid_types:
                    raise ValueError(f"Invalid type: {t}. Must be one of {valid_types}")
        return v

In [4]:
#| export
class Transform(Protocol):
    """Protocol for feature transformations"""
    def __call__(self, df: DataFrame) -> DataFrame: ...
    
    @property
    def config(self) -> TransformConfig: ...

In [5]:
#| export
class ValidationMixin:
    """Mixin class providing validation methods for transforms"""
    def validate_dataframe(self, df: DataFrame, columns: List[str]) -> None:
        """Validate DataFrame columns against configuration"""
        if not hasattr(self, 'config'):
            return
            
        try:
            # Check for nulls
            if self.config.null_threshold < 1.0:
                for col in columns:
                    null_count = df.filter(F.col(col).is_null()).count()
                    total_count = df.count()
                    null_ratio = null_count / total_count if total_count > 0 else 0
                    
                    if null_ratio > self.config.null_threshold:
                        raise ValidationError(
                            f"Column {col} has {null_ratio:.1%} null values, "
                            f"exceeding threshold of {self.config.null_threshold:.1%}"
                        )
                        
            # Check cardinality
            if self.config.cardinality_threshold:
                for col in columns:
                    distinct_count = df.select(col).distinct().count()
                    if distinct_count > self.config.cardinality_threshold:
                        raise ValidationError(
                            f"Column {col} has {distinct_count} distinct values, "
                            f"exceeding threshold of {self.config.cardinality_threshold}"
                        )
                        
            # Check data types if specified
            if self.config.expected_types:
                for col in columns:
                    col_type = str(df.schema[col].datatype)
                    valid_type = any(
                        col_type.upper().startswith(exp_type.upper()) 
                        for exp_type in self.config.expected_types
                    )
                    if not valid_type:
                        raise ValidationError(
                            f"Column {col} has type {col_type}, "
                            f"expected one of {self.config.expected_types}"
                        )
                        
            logger.debug(f"Validation passed for columns: {columns}")
            
        except Exception as e:
            if not isinstance(e, ValidationError):
                raise ValidationError(f"Validation failed: {str(e)}")
            raise

In [6]:
#| export
class CustomTransform(ValidationMixin):
    """Wrapper for custom transformations"""
    def __init__(
        self,
        transform_func: Callable[[DataFrame], DataFrame],
        config: TransformConfig
    ):
        self._transform = transform_func
        self._config = config
        
    @property
    def config(self) -> TransformConfig:
        return self._config
        
    def __call__(self, df: DataFrame) -> DataFrame:
        return self._transform(df)

In [7]:
#| export
@dataclass
class WindowSpec:
    """Configuration for window-based transformations"""
    partition_by: Optional[Union[str, List[str]]] = None
    order_by: Optional[Union[str, List[str]]] = None
    window_size: Optional[int] = None
    
    def __post_init__(self):
        """Convert string inputs to lists"""
        if isinstance(self.partition_by, str):
            self.partition_by = [self.partition_by]
        if isinstance(self.order_by, str):
            self.order_by = [self.order_by]
            
    def to_window(self) -> Window:
        """Convert to Snowpark Window specification"""
        window = Window.partition_by(self.partition_by or []) \
                      .order_by(self.order_by or [])
        
        if self.window_size:
            window = window.rows_between(
                -(self.window_size-1),
                Window.CURRENT_ROW
            )
        return window

In [8]:
#| export
class WindowTransform(ValidationMixin):
    """Window-based aggregation transform"""
    
    def __init__(
        self,
        agg_cols: Dict[str, List[str]],
        window_spec: WindowSpec,
        config: Optional[TransformConfig] = None
    ):
        self._agg_cols = agg_cols
        self._window_spec = window_spec
        self._config = config or TransformConfig(
            name="window_transform",
            description="Window-based aggregation"
        )
        
    @property
    def config(self) -> TransformConfig:
        return self._config
        
    def __call__(self, df: DataFrame) -> DataFrame:
        """Apply window-based aggregations"""
        try:
            window = self._window_spec.to_window()
            new_cols = []
            
            for col, aggs in self._agg_cols.items():
                for agg in aggs:
                    agg_func = getattr(F, agg.lower())
                    new_col = f"{agg.upper()}_{col.upper()}"
                    new_cols.append(new_col)
                    df = df.with_column(
                        new_col,
                        agg_func(F.col(col)).over(window)
                    )
                    
            if self.config.validate_output:
                self.validate_dataframe(df, new_cols)
                
            return df
            
        except Exception as e:
            logger.error(f"Window transform failed: {str(e)}")
            raise

In [9]:
#| export
def window_agg(
    agg_cols: Dict[str, List[str]],
    window_spec: WindowSpec,
    config: Optional[TransformConfig] = None
) -> Transform:
    """Create window-based aggregation transform
    
    Args:
        agg_cols: Dictionary mapping columns to aggregation functions
        window_spec: Window specification for the aggregation
        config: Optional transform configuration
        
    Returns:
        Transform function
        
    Example:
        >>> config = TransformConfig(
        ...     name="customer_metrics",
        ...     null_threshold=0.05,
        ...     expected_type="DOUBLE"
        ... )
        >>> spec = WindowSpec(partition_by='customer_id', order_by='date')
        >>> transform = window_agg({'amount': ['SUM', 'AVG']}, spec, config)
    """
    return WindowTransform(agg_cols, window_spec, config)

In [10]:
#| export
class FillNATransform(ValidationMixin):
    def __init__(
        self, 
        cols: Union[str, List[str]], 
        fill_value: Union[int, float, str] = 0,
        config: Optional[TransformConfig] = None
    ):
        self.cols = listify(cols)
        self.fill_value = fill_value
        self._config = config or TransformConfig(
            name="fill_na_transform",
            description=f"Fill NA values with {fill_value}"
        )
        
    @property
    def config(self) -> TransformConfig:
        return self._config
        
    def __call__(self, df: DataFrame) -> DataFrame:
        try:
            for col in self.cols:
                actual_col = next(
                    (c for c in df.schema.names if c.upper() == col.upper()),
                    None
                )
                if actual_col is None:
                    raise ValueError(f"Column {col} not found in DataFrame")
                
                col_type = df.schema[actual_col].datatype
                typed_value = (
                    int(self.fill_value) if str(col_type).startswith(('Long', 'Int')) 
                    else float(self.fill_value) if str(col_type).startswith('Double') 
                    else str(self.fill_value)
                )
                
                df = df.na.fill({actual_col: typed_value})
                
            if self.config.validate_output:
                self.validate_dataframe(df, self.cols)
                
            return df
            
        except Exception as e:
            logger.error(f"Fill NA transform failed: {str(e)}")
            raise

In [11]:

#| export
def fill_na(
    cols: Union[str, List[str]], 
    fill_value: Union[int, float, str] = 0,
    config: Optional[TransformConfig] = None
) -> Transform:
    """Create transform to fill NA values
    
    Args:
        cols: Column(s) to fill NA values in
        fill_value: Value to use for filling NAs
        config: Optional transform configuration
        
    Returns:
        Transform function
        
    Example:
        >>> config = TransformConfig(
        ...     name="fill_scores",
        ...     null_threshold=1.0
        ... )
        >>> transform = fill_na(['score'], fill_value=0, config=config)
    """
    return FillNATransform(cols, fill_value, config)


In [12]:
# | export
class DateDiffTransform(ValidationMixin):
    """Calculate date difference between column and reference"""
    
    def __init__(
        self,
        col: str,
        new_col: str,
        reference_date: Optional[str] = None,
        date_part: str = 'day',
        config: Optional[TransformConfig] = None
    ):
        self.col = col
        self.new_col = new_col.upper()
        self.reference_date = reference_date
        self.date_part = date_part
        self._config = config or TransformConfig(
            name="date_diff_transform",
            description=f"Calculate {date_part} difference from {col}",
            expected_type="INT"  # Date diffs return integers
        )
        
    @property
    def config(self) -> TransformConfig:
        return self._config
        
    def __call__(self, df: DataFrame) -> DataFrame:
        try:
            # Find actual column name (case-insensitive)
            col_actual = next(
                (c for c in df.columns if c.upper() == self.col.upper()),
                None
            )
            if col_actual is None:
                raise ValueError(f"Column {self.col} not found in DataFrame")
            
            reference = (F.to_date(F.lit(self.reference_date)) 
                        if self.reference_date
                        else F.current_date())
            
            df = df.with_column(
                self.new_col,
                F.datediff(self.date_part, F.col(col_actual), reference)
            )
            
            if self.config.validate_output:
                self.validate_dataframe(df, [self.new_col])
                
            return df
            
        except Exception as e:
            logger.error(f"Date diff transform failed: {str(e)}")
            raise



In [13]:
#| export
def date_diff(
    col: str,
    new_col: str,
    reference_date: Optional[str] = None,
    date_part: str = 'day',
    config: Optional[TransformConfig] = None
) -> Transform:
    """Create date difference transform
    
    Args:
        col: Date column to calculate difference from
        new_col: Name for the new difference column
        reference_date: Reference date (default: current_date)
        date_part: Part to calculate difference in ('day', 'month', etc.)
        config: Optional transform configuration
        
    Returns:
        Transform function
        
    Example:
        >>> config = TransformConfig(
        ...     name="membership_length",
        ...     null_threshold=0.0  # No nulls allowed
        ... )
        >>> transform = date_diff('join_date', 'days_member', config=config)
    """
    return DateDiffTransform(col, new_col, reference_date, date_part, config)

In [14]:
#| export
class MovingAggTransform(ValidationMixin):
    """Calculate moving window aggregations"""
    
    def __init__(
        self,
        cols: Union[str, List[str]],
        window_sizes: List[int],
        agg_funcs: List[str] = ['SUM', 'AVG'],
        partition_by: Optional[List[str]] = None,
        order_by: Optional[List[str]] = None,
        config: Optional[TransformConfig] = None
    ):
        self.cols = listify(cols)
        self.window_sizes = window_sizes
        self.agg_funcs = agg_funcs
        self.partition_by = partition_by
        self.order_by = order_by
        self._config = config or TransformConfig(
            name="moving_agg_transform",
            description=f"Moving {', '.join(agg_funcs)} over {window_sizes} periods"
        )
        
    @property
    def config(self) -> TransformConfig:
        return self._config
        
    def __call__(self, df: DataFrame) -> DataFrame:
        try:
            new_cols = []
            
            for col in self.cols:
                for size in self.window_sizes:
                    spec = WindowSpec(
                        partition_by=self.partition_by,
                        order_by=self.order_by,
                        window_size=size
                    )
                    
                    window = spec.to_window()
                    
                    for agg in self.agg_funcs:
                        agg_func = getattr(F, agg.lower())
                        new_col = f"{agg.upper()}_{col.upper()}_{size}"
                        new_cols.append(new_col)
                        
                        # Cast result to DOUBLE
                        df = df.with_column(
                            new_col,
                            agg_func(F.col(col)).over(window).cast('DOUBLE')
                        )
            
            if self.config.validate_output:
                self.validate_dataframe(df, new_cols)
                
            return df
            
        except Exception as e:
            logger.error(f"Moving aggregation transform failed: {str(e)}")
            raise


In [15]:
#| export

def moving_agg(
    cols: Union[str, List[str]],
    window_sizes: List[int],
    agg_funcs: List[str] = ['SUM', 'AVG'],
    partition_by: Optional[List[str]] = None,
    order_by: Optional[List[str]] = None,
    config: Optional[TransformConfig] = None
) -> Transform:
    """Create moving aggregation transform
    
    Args:
        cols: Columns to aggregate
        window_sizes: List of window sizes
        agg_funcs: List of aggregation functions
        partition_by: Columns to partition by
        order_by: Columns to order by
        config: Optional transform configuration
        
    Returns:
        Transform function
        
    Example:
        >>> config = TransformConfig(
        ...     name="rolling_metrics",
        ...     expected_type="DOUBLE"
        ... )
        >>> transform = moving_agg(
        ...     'amount', 
        ...     [7, 30], 
        ...     ['SUM'], 
        ...     ['customer_id'], 
        ...     ['date'],
        ...     config
        ... )
    """
    return MovingAggTransform(
        cols, window_sizes, agg_funcs, 
        partition_by, order_by, config
    )

In [16]:
#| export
class CumulativeAggTransform(ValidationMixin):
    """Calculate cumulative aggregations"""
    
    def __init__(
        self,
        cols: Union[str, List[str]],
        agg_funcs: List[str] = ['SUM'],
        partition_by: Optional[List[str]] = None,
        order_by: Optional[List[str]] = None,
        config: Optional[TransformConfig] = None
    ):
        self.cols = listify(cols)
        self.agg_funcs = agg_funcs
        self.partition_by = partition_by
        self.order_by = order_by
        self._config = config or TransformConfig(
            name="cumulative_agg_transform",
            description=f"Cumulative {', '.join(agg_funcs)}"
        )
        
    @property
    def config(self) -> TransformConfig:
        return self._config
        
    def __call__(self, df: DataFrame) -> DataFrame:
        try:
            new_cols = []
            
            for col in self.cols:
                spec = WindowSpec(
                    partition_by=self.partition_by,
                    order_by=self.order_by
                )
                window = spec.to_window()
                
                for agg in self.agg_funcs:
                    agg_func = getattr(F, agg.lower())
                    new_col = f"CUM_{agg.upper()}_{col.upper()}"
                    new_cols.append(new_col)
                    
                    df = df.with_column(
                        new_col,
                        agg_func(F.col(col)).over(window)
                    )
            
            if self.config.validate_output:
                self.validate_dataframe(df, new_cols)
                
            return df
            
        except Exception as e:
            logger.error(f"Cumulative aggregation transform failed: {str(e)}")
            raise

In [17]:
#| export
def cumulative_agg(
    cols: Union[str, List[str]],
    agg_funcs: List[str] = ['SUM'],
    partition_by: Optional[List[str]] = None,
    order_by: Optional[List[str]] = None,
    config: Optional[TransformConfig] = None
) -> Transform:
    """Create cumulative aggregation transform
    
    Args:
        cols: Columns to aggregate
        agg_funcs: List of aggregation functions
        partition_by: Columns to partition by
        order_by: Columns to order by
        config: Optional transform configuration
        
    Returns:
        Transform function
        
    Example:
        >>> config = TransformConfig(
        ...     name="running_totals",
        ...     expected_type="DOUBLE"
        ... )
        >>> transform = cumulative_agg(
        ...     'amount', 
        ...     ['SUM'], 
        ...     ['customer_id'], 
        ...     ['date'],
        ...     config
        ... )
    """
    return CumulativeAggTransform(
        cols, agg_funcs, partition_by, order_by, config
    )

In [18]:
#| export
def apply_transforms(df: DataFrame, transforms: List[Transform]) -> DataFrame:
    """Apply a list of transformations to a DataFrame
    
    Args:
        df: Input DataFrame
        transforms: List of transform functions to apply
        
    Returns:
        Transformed DataFrame
        
    Example:
        >>> transforms = [
        ...     fill_na(['score'], config=TransformConfig(name='fill_scores')),
        ...     date_diff('date', 'days_ago', config=TransformConfig(name='time_features'))
        ... ]
        >>> df = apply_transforms(df, transforms)
    """
    for transform in transforms:
        try:
            logger.debug(f"Applying transform: {transform.config.name}")
            df = transform(df)
        except Exception as e:
            logger.error(f"Transform {transform.config.name} failed: {str(e)}")
            raise
    return df


In [19]:
#| hide
import nbdev; nbdev.nbdev_export()

In [20]:
#| eval: false
# Example usage
from snowflake_feature_store.connection import get_connection, ConnectionConfig
from snowflake_feature_store.transforms import *

# Get connection automatically
conn = get_connection()
session = conn.session

# Create example DataFrame - Note the UPPERCASE column names for Snowflake
df = session.create_dataframe([
    ['C1', '2024-01-01', 100, None],
    ['C1', '2024-01-02', 200, 3.5],
    ['C2', '2024-01-01', 150, None]
], ['CUSTOMER_ID', 'DATE', 'AMOUNT', 'SCORE'])

# Configuration with appropriate type validation
agg_config = TransformConfig(
    name="customer_metrics",
    null_threshold=0.05,
    expected_types=['LONG', 'DOUBLE']  # Accept either type for numeric aggregations
)

date_config = TransformConfig(
    name="date_features",
    null_threshold=0.0,
    expected_types=['INT', 'LONG']  # Date diffs can be either type
)

# Create transforms
transforms = [
    # Fill NA values in score column
    fill_na(['SCORE'], fill_value=0),
    
    # Calculate moving aggregations on amount
    moving_agg(
        cols='AMOUNT',
        window_sizes=[2],
        agg_funcs=['SUM', 'AVG'],  # Now we can use both
        partition_by=['CUSTOMER_ID'],
        order_by=['DATE'],
        config=agg_config
    ),
    
    # Calculate date differences
    date_diff(
        col='DATE',
        new_col='DAYS_AGO',
        reference_date='2024-01-03',
        config=date_config
    )
]

# Apply transforms
result_df = apply_transforms(df, transforms)

# Show results
print("\nTransformed DataFrame:")
result_df.show()

# Show schema
print("\nResult Schema:")
for field in result_df.schema.fields:
    print(f"{field.name}: {field.datatype}")


2025-02-26 19:34:47,083 - snowflake_feature_store - INFO - No active session found, creating new connection from environment
2025-02-26 19:34:47,762 - snowflake_feature_store - INFO - Initialized connection to "CONTAINER_DEMO_DB"."PUBLIC"
2025-02-26 19:34:47,763 - snowflake_feature_store - INFO - Using role: "ACCOUNTADMIN", warehouse: "CONTAINER_DEMO_WH", database: "CONTAINER_DEMO_DB", schema: "PUBLIC"

Transformed DataFrame:
--------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"DATE"      |"AMOUNT"  |"SCORE"  |"SUM_AMOUNT_2"  |"AVG_AMOUNT_2"  |"DAYS_AGO"  |
--------------------------------------------------------------------------------------------------
|C2             |2024-01-01  |150       |0.0      |150.0           |150.0           |2           |
|C1             |2024-01-01  |100       |0.0      |100.0           |100.0           |2           |
|C1             |2024-01-02  |200       |3.5      |300.0           |150.0  